# Scraping trail running races 2021

This notebook scraps the trail running races' data from https://itra.run/races. Environment preparation is managed by the following scripts:
- WindowsEnvironment.ps1
- MacOSEnvironment.sh

In [ ]:
import logging
# Logging format
format = "%(asctime)s: %(message)s"
logging.basicConfig(filename='PRA1.log', format=format, level=logging.INFO,datefmt="%H:%M:%S")
#logging.basicConfig(format=format, level=logging.INFO,datefmt="%H:%M:%S")
logging.info("Log started")

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

import concurrent.futures
import time
import re
import json
import pandas as pd
import whois
import sys
import datetime
from tqdm import tqdm
from datetime import timedelta
import requests
import json


Per tal de ser més eficients, crarem diferents instàncies d'exploradors que facin la mateixa feina però amb rangs de dates diferents. Primer definim una funció que permet partir un període de temps en subperiodes.

In [ ]:
def splitTimePeriod(n, start, end):
    """
    splitTimePeriod gets an array of dates [n,2] with the period splatted in n subperiods

    :param n: number of subperiods
    :param start: Start date 
    :param end: End date 
    :return: returns a [n,2] matrix with the subperiods dates
    """     
    logging.info("splitTimePeriod %s / %s period in %s subperiods: starting", start, end, n)

    duration = end - start

    durationDays = duration.total_seconds() / 60 / 60 / 24
    logging.info("splitTimePeriod %s / %s period in %s subperiods: total duration in days: %s", start, end, n, durationDays)

    subperiodDuration = 1
    
    if n != 0:
        subperiodDuration = durationDays / n

    logging.info("splitTimePeriod %s / %s period in %s subperiods: subperiod duration in days: %s", start, end, n, subperiodDuration)

    result = []
    
    if subperiodDuration < 2:
        # Too small period return only one array
        result = [[start,end]]
    else:
        # Split the period
        init = start
        
        # Remove decimals
        periodDays = int(subperiodDuration)

        for x in range(n-1):
            result.append([init,init+timedelta(periodDays-1)])
            init = init+timedelta(periodDays)

        #Last subperiod
        result.append([init,end])
    
    return result    

Definim també una funció per a obtenir les dades que ens interessen amb la llibreria BeautifulSoup.

In [ ]:
def getData(n, htmlSource):
    """
    getData gets the races from ITRA in the given period time

    :param htmlSource: Source HTML for the apge 
    :return: data
    """
    logging.info("Thread %s: Getting Data", n) 
    
    # Scraping race names with BeautifulSoup
    soup = BeautifulSoup(htmlSource, 'html')
    logging.info("Thread %s: html extracted", n) 
    
    # Find the race names
    racesList = re.findall(r'(?<=<h5 data-v-f3c4ac1c="" class="itra-green">)(.*?)(?=</h5>)', htmlSource)
    logging.info("Thread %s: Number of races %s",n,len(racesList))
    #print(racesList)

    # Find the mouse over link pointing to the race site
    links = [a['href'] for a in soup.find_all('a',"card ontop", href=True)]
    logging.info("Thread %s: Number of links %s",n,len(links))

    # Scraping the data for distance, elevation gain and loss
    myList = re.findall(r'(?<=<span class="icon-text-grey icon-bold">)(.*?)(?=</span>)', htmlSource)
    logging.info("Thread %s: Number of measures %s",n,len(myList))

    # Find the race distance
    distancesList = myList[0::3]
    
    # Find the race elevation gain
    gainList = myList[1::3]
    
    # Find the race elevation loss
    lossList = myList[2::3]
    
    # Find the race date
    datesList = re.findall(r'(?<=<span data-v-f3c4ac1c="" class="itra-grey" style="margin-top: 0.2rem; margin-left: 0.2rem; margin-right: 2rem; font-size: 80%;">)(.*?)(?=</span>)', htmlSource)
    #print(datesList)
    logging.info("Thread %s: Number of races %s",n,len(datesList))

    # Assign data to tuples: # get the list of tuples from two lists and merge them by using zip(). 
    list_of_tuples = list(zip(racesList, links, distancesList, gainList, lossList, datesList)) 
    # Converting lists of tuples into pandas Dataframe. 
    df = pd.DataFrame(list_of_tuples, columns = ['Name', 'Link', 'Distance', 'Gain', 'Loss', 'Date'])
    logging.info("Thread %s: Get Data terminated", n) 
    return df

Ara definim una nova funció que obtingui amb selenium el codi font de la plana web de curses. 

In [ ]:
def getRaces(n, period):
    """
    getRaces gets the races from ITRA in the given period time

    :param n: Thread context execution
    :param period: Start and End date 
    :return: All races with high level data for the given period
    """ 
    logging.info("Thread %s for %s period: starting getRaces", n, period)
    
    time.sleep(2)
    
    # Using selenium, open firefox window with the ITRA website
    driver = webdriver.Firefox()
    driver.get("https://itra.run/races")

    # Getting current URL source code 
    get_title = driver.title 

    # Click dropdown menu for language selection
    driver.find_element_by_xpath("/html/body/div[1]/div[1]/nav/div[4]").click()

    # Select language EN
    driver.find_element_by_xpath("/html/body/div[1]/div[1]/nav/div[4]/div/div[1]").click()

    # Wait for page load after click
    time.sleep(5)

    # Select init date
    initYear=period[0].year
    initMonth=period[0].month
    initDay=period[0].day

    # Picker de la data d'inici
    dpkStartdate = driver.find_element_by_css_selector("div.vdp-datepicker.dp1")
    dpkStartdate.click()

    # Picker per mostrar anys
    spnMonthSelector = driver.find_element_by_css_selector(".dp1 span.day__month_btn.up")
    spnMonthSelector.click()

    # Picker per mostrar anys
    spnYearSelector = driver.find_element_by_css_selector(".dp1 span.month__year_btn.up")
    spnYearSelector.click()

    # Picker per seleccionar any
    divYears = driver.find_elements_by_css_selector(".dp1 span.cell.year")
    divYears[initYear-2020].click()

    # Picker per seleccionar mes
    divMonths = driver.find_elements_by_css_selector(".dp1 span.cell.month")
    divMonths[initMonth-1].click()

    # Picker per seleccionar dia
    divDays = driver.find_elements_by_css_selector(".dp1 span.cell.day")
    # Get the days from the previous month in the current mont first week
    divBlankDays = driver.find_elements_by_css_selector(".dp1 span.cell.day.blank")
    divDays[initDay+len(divBlankDays)-1].click()  

    # Select end date
    endYear=period[1].year
    endMonth=period[1].month
    endDay=period[1].day

    # Picker de la data d'inici
    dpkStartdate = driver.find_element_by_css_selector("div.vdp-datepicker.dp2")
    dpkStartdate.click()

    # Picker per mostrar anys
    spnMonthSelector = driver.find_element_by_css_selector(".dp2 span.day__month_btn.up")
    spnMonthSelector.click()

    # Picker per mostrar anys
    spnYearSelector = driver.find_element_by_css_selector(".dp2 span.month__year_btn.up")
    spnYearSelector.click()

    # Picker per seleccionar any
    divYears = driver.find_elements_by_css_selector(".dp2 span.cell.year")
    divYears[endYear-2020].click()

    # Picker per seleccionar mes
    divMonths = driver.find_elements_by_css_selector(".dp2 span.cell.month")
    divMonths[endMonth-1].click()

    # Picker per seleccionar dia
    divDays = driver.find_elements_by_css_selector(".dp2 span.cell.day")
    # Get the days from the previous month in the current mont first week
    divBlankDays = driver.find_elements_by_css_selector(".dp2 span.cell.day.blank")
    divDays[endDay+len(divBlankDays)-1].click()

    # Wait for page load
    time.sleep(5)  

    # Get the number of total races for the given subperiod
    logging.info("Thread %s: Retrieving number of races",n) 
    totalRacesText = re.findall(r'(?<=<h1 class="itra-green text-center">)([0-9]+)(?=.*</h1>)', driver.page_source)
    totalRaces = int(totalRacesText[0])
    logging.info("Thread %s: Number of races %s",n,totalRaces)    

    # Initialize result
    resultPeriod = []

    if totalRaces > 0:

        # Click on More Races to get the full list on the screen & Wait for Visibility of Races until we list all races
        maxIterations = int(totalRaces / 50)

        logging.info("Thread %s: We will use %s iterations for %s races",n,maxIterations,totalRaces)    

        if totalRaces % 50 > 0:
            maxIterations = maxIterations + 1

        i = 0
        try:
            logging.info("Thread %s for %s period: %s iteration", period, n, i)
            btnSeeMore = driver.find_element_by_css_selector('button.btn-itra-black[type="button"]')    

        except:
            btnSeeMore = None
            logging.info("Thread %s for %s period: No more races", period, n)

        while btnSeeMore is not None and i != maxIterations-1:
            i = i+1        

            try:
                btnSeeMore.click()
                logging.info("Thread %s for %s period: %s iteration", period, n, i)
                btnSeeMore = driver.find_element_by_css_selector('button.btn-itra-black[type="button"]') 
                
            except:            
                btnSeeMore = None
                logging.info("Thread %s for %s period: No more races", period, n)
                break

            finally:
                time.sleep(5)
        
        # Obtnim les dades amb BaeatifulSoup 
        resultPeriod = getData(n, driver.page_source)
        logging.info("Thread %s for %s period: Data frame returned with %s records", period, n, len(resultPeriod))
    else:
        #Generate a dataframe empty with the same structure
        list_of_tuples = list(zip([], [], [], [], [], [])) 
        resultPeriod = pd.DataFrame(list_of_tuples, columns = ['Name', 'Link', 'Distance', 'Gain', 'Loss', 'Date'])
    driver.quit()
    return resultPeriod

Definim una funció que obten les curses limitant el número de dies del periode a consultar. En cas que hi hagi més dies executa el procés de forma iterativa per evitar problemes de rendiment i estabilitat en execucions massa llargues

In [ ]:
def getRacesSeq(n, period, maxDuration=30):
    """
    getRacesSeq gets the races from ITRA in the given period time. Opens a new browser session for each maxDuration days to avoid issues with the borwser when too much races needs to bre retrieved.

    :param n: Thread context execution
    :param period: Start and End date 
    :param maxDuration: Maximum days to query to the browser
    :return: All races with high level data for the given period
    """ 
    logging.info("Thread %s for %s period: starting getRacesSeq", n, period)

    # Get the number of subperiods
    duration = period[1] - period[0]
    durationDays = int(duration.total_seconds() / 60 / 60 / 24)
    
    logging.info("Thread %s for %s period: Period duration is %s days", n, period,durationDays)
    subperiodsSeq = splitTimePeriod(int(durationDays/maxDuration), period[0], period[1])
    logging.info("Thread %s for %s period: Periods %s", n, period,subperiodsSeq)
    
    i = 0
    result = []

    while i < len(subperiodsSeq):
        logging.info("Thread %s for %s period: Processing period", n, subperiodsSeq[i])
        resultAux = getRaces(n,subperiodsSeq[i])
        logging.info("Thread %s for %s period: Processed period with %s races", n, subperiodsSeq[i], len(resultAux))
        result.append(resultAux)
        i = i + 1
    result  = pd.concat(result)
    logging.info("Thread %s for %s period: terminated getRacesSeq with %s records", n, period, len(result))
    return result

Amb les funcions anteriors podem fer l'execució on inicialitzem les variables d'entrada.

In [ ]:
# Period definition
startDate = datetime.datetime(2020, 1, 1, 0, 0, 0)
endDate = datetime.datetime(2022, 6, 30, 0, 0, 0)

# Number of threads
threads = 3

# Get periods
periods = splitTimePeriod(threads, startDate, endDate)

# Parallel execution
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    resultPeriods = list(executor.map(getRacesSeq, range(threads),periods))

# Concatene all results
resultFullPeriod = pd.concat(resultPeriods)
logging.info("Races for period %s to %s is %s", startDate, endDate, len(resultFullPeriod))
print(resultFullPeriod)


Obtenim els identificadors únics de les curses

In [ ]:
# Remove unnecessary part of the string
sep = 'https://itra.run/race/'
id_unic =[item.split(sep, 1)[1] for item in resultFullPeriod['Link'].tolist()]

# Removing single quotes (effectively transforms it to int)
id_unic_int=[int(x) for x in id_unic]

# Adding to the dataset
resultFullPeriod['id_course'] = id_unic_int

Ara ataquem les dades de la pàgina específica de cada cursa

In [ ]:
# Extraccio de dades de les planes detallades
import requests
import json

# Removing single quotes (effectively transforms it to int)
id_unic_int=[int(x) for x in id_unic]

# Remove duplicates
id_unic_int=set(id_unic_int)
id_unic_int=list(id_unic_int)

# List of json containing datailed info on each race
jsonList = []

# Add year in the dataset as the PK for a race is id and year
resultFullPeriod['Year'] = resultFullPeriod['Date'].str[-4:].astype(int)

# List of json containing datailed info on each race
cursesList = []

# Loop over the urls to obtain details in json
for item in id_unic_int:
    
    try:
        # Get the years of the course
        years = len(resultFullPeriod.loc[resultFullPeriod['id_course'] == item])

        i = 0

        while i < years:
            # get the year of the course
            year = (resultFullPeriod.loc[resultFullPeriod['id_course'] == item]).iloc[i]['Year']
            
            # url and request
            url = "https://itra.run/evt/racewdetails/"
            final_url = f'{url}{item}/{year}'
            
            logging.info("Requesting %s",final_url)
            r = requests.get(final_url, verify=False)
            logging.info("Requested %s with HTTP status %s",final_url,r.status_code)
            
        
            # Add to list of json races
            cursa=r.json()[0]
            jsonList.append(cursa)
            
            # Add to dataframe of races    
            df_cursa = pd.DataFrame([cursa])
            cursesList.append(df_cursa)

            i = i + 1
    except:
        logging.error("Error processing race with %s (%s)",final_url,sys.exc_info()[0])


# Dataframe detailed race information
df_detailed = pd.concat(cursesList)
df_detailed=df_detailed.reset_index(drop=True)

# Concatenate df & df_detailed by column
resultFullPeriod = pd.merge(resultFullPeriod, df_detailed, left_on=['id_course','Year'], right_on=['id_course','annee'])
logging.info("Processed all %s races in the period %s - %s", len(resultFullPeriod),startDate,endDate)
        

Eliminem les columnes que no ens interessen i guardem el fitxer en format CSV

In [ ]:
# Remove columns
del resultFullPeriod["Gain"]
del resultFullPeriod["Loss"]
del resultFullPeriod["dt"]
del resultFullPeriod["descr"]
del resultFullPeriod["descr_en"]
del resultFullPeriod["nb_eqp"]
del resultFullPeriod["nb_pts_tps"]
del resultFullPeriod["type_course"]
del resultFullPeriod["inscr_cond"]
del resultFullPeriod["brouillon"]
del resultFullPeriod["dist_tot2"]
del resultFullPeriod["ravitos"]

# Save the file
resultFullPeriod.to_csv("Trail Running Races 2020-2029.csv",index=True)

In [ ]:
print(resultFullPeriod.dtypes)

In [ ]:
print(df_detailed.dtypes)